In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from text_token import column_token
from gensim.models import word2vec

pd.options.display.max_colwidth = 200

In [2]:
df = pd.read_csv("data/final_sub.csv")
#df = pd.read_csv("data/final_full.csv")

In [3]:
df_train = pd.read_csv('data/final_train_sub.csv')
#df_train = pd.read_csv('data/final_train_full.csv')

df_test = pd.read_csv('data/final_test_sub.csv')
#df_test = pd.read_csv('data/final_test_full.csv')

In [4]:
df1_train = column_token(df_train, 'reviewText')
df1_test = column_token(df_test, 'reviewText')
df1_train.head()

,Unnamed: 0,label,reviewText,clean_text
0,4227,1,"Wow, just finished reading this book it was different and interesting but at the same time passionable and great love.","[wow, just, finished, reading, book, different, interesting, time, passionable, great, love]"
1,4676,0,How many stereotypes can be crammed into 1 short story. I got this since it appeared to be a modern continuation of &#34;To Ride a Puka&#34; I highly recommend that book. But don't bother with thi...,"[many, stereotypes, can, crammed, short, story, got, since, appeared, modern, continuation, ride, puka, highly, recommend, book, don, t, bother, one]"
2,800,1,OMG. I can't get enough of this series. Just love Taron and Ivy. I love the Walker brothers. Can't wait to see whats happens next.,"[omg, can, t, get, enough, series, just, love, taron, ivy, love, walker, brothers, can, t, wait, see, whats, happens, next]"
3,3671,1,Wow! You will need a tissue when you read this. Poor Alex has so many things happen to her in her 30 years. The bulk of the story takes place while she is 15. Fast forward to present day with a se...,"[wow, will, need, tissue, read, poor, alex, many, things, happen, years, bulk, story, takes, place, fast, forward, present, day, second, chance, love, one, deserves, happiness, much, alex, sweet, ..."
4,4193,1,"Set in LA, there are hints of LA Confidential along with a quirky heroine and a solid mystery. ""Kenny"" Rubin (the name is short for a name she hates) is a woman in a tough town. She also writes fo...","[set, la, hints, la, confidential, along, quirky, heroine, solid, mystery, kenny, rubin, name, short, name, hates, woman, tough, town, also, writes, magazine, s, fold, s, covering, five, year, old..."


In [5]:
df_train.shape

(4000, 4)

In [6]:
from doc_embed import averaged_word_vectorizer

In [7]:
# Set values for various parameters
feature_size = 10    # Word vector dimensionality  
window_context = 10          # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(df1_train['clean_text'], size=feature_size, 
                              window=window_context, min_count = min_word_count,
                              sample=sample, iter=100)

In [8]:
w2v_feature_array_train = averaged_word_vectorizer(corpus=df1_train['clean_text'], model=w2v_model,
                                             num_features=feature_size)
X_train = pd.DataFrame(w2v_feature_array_train)

In [9]:
w2v_feature_array_test = averaged_word_vectorizer(corpus=df1_test['clean_text'], model=w2v_model,
                                             num_features=feature_size)
X_test = pd.DataFrame(w2v_feature_array_test)

In [10]:
y_train = df1_train['label']
y_test = df1_test['label']

In [11]:
#X_train.head()

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

/Users/louislimnavong/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
model = GradientBoostingClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)

In [14]:
print(confusion_matrix(y_train, model.predict(X_train)))

[[1764  150]
 [ 131 1955]]


In [15]:
print(confusion_matrix(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))
print(accuracy_score(y_test, model.predict(X_test)))

[[421  57]
 [ 44 478]]
             precision    recall  f1-score   support

          0       0.91      0.88      0.89       478
          1       0.89      0.92      0.90       522

avg / total       0.90      0.90      0.90      1000

0.899


# Diagnosis

In [16]:
df1_test['predict_label'] =  model.predict(X_test)
df1_test['Prediction'] =  model.predict_proba(X_test)[:,0]
df1_test.head()

,Unnamed: 0,label,reviewText,clean_text,predict_label,Prediction
0,1501,1,"There&#8217;s (almost!) nothing I enjoy more than reading a book that whisks me away from reality and submerges me in a fantasy world filled with good, evil and steamy seduction! In this book, our...","[s, almost, nothing, enjoy, reading, book, whisks, away, reality, submerges, fantasy, world, filled, good, evil, steamy, seduction, book, heroine, melisse, lowly, servant, humiliated, every, turn,...",1,0.062223
1,2586,0,This was an amateurish attempt at writing. The dialogue was stilted and the plot predictable. This story completed lacked originality and seems like a poorly written short story for someone in a...,"[amateurish, attempt, writing, dialogue, stilted, plot, predictable, story, completed, lacked, originality, seems, like, poorly, written, short, story, someone, writing, class, nothing, series, th...",0,0.990885
2,2653,0,"This was a poorly written piece that I wish I had not wasted my money on. Apart from using the word than when he should use then, the author also constantly made sweeping generalisations that wer...","[poorly, written, piece, wish, wasted, money, apart, using, word, use, author, also, constantly, made, sweeping, generalisations, just, ludicrous]",0,0.980523
3,1055,0,Perhaps if I had young children and was looking for a projects to do together this would have been interesting.,"[perhaps, young, children, looking, projects, together, interesting]",1,0.113279
4,705,0,"Details. . I chose the book in a bored state. Too much sexually explicit, by a dreamer of what she could not actually experience so she may climax in her sick soul.","[details, chose, book, bored, state, much, sexually, explicit, dreamer, actually, experience, may, climax, sick, soul]",0,0.940147


In [17]:
df1_test['error'] = abs(df1_test['label'] - df1_test['predict_label'])

In [18]:
df_temp = df1_test[['reviewText','clean_text','label', 'predict_label', 'Prediction']][(df1_test['error'] == 1) & (df1_test['predict_label'] == 0)].sort_values(by='Prediction', ascending = False).head()
df_temp.head()



,reviewText,clean_text,label,predict_label,Prediction
542,I will say this was a good end to this series. What I didn't like was the fact was the book only had a hundred and something pages for that price.,"[will, say, good, end, series, didn, t, like, fact, book, hundred, something, pages, price]",1,0,0.963726
431,"Since I don't have wi-fi for my Kindle, this USB Cable lets me download books, etc. from my PC to my Kindle.","[since, don, t, wi, fi, kindle, usb, cable, lets, download, books, etc, pc, kindle]",1,0,0.948436
225,I have to give credit where it's due!! I'm only upset that part III isn't out & there's no mention of when to expect it!! But don't worry I'm a fan for life so please hurry with the follow up!!,"[give, credit, s, due, m, upset, part, iii, isn, t, s, mention, expect, don, t, worry, m, fan, life, please, hurry, follow]",1,0,0.945464
942,Because I didn't have any problems with my version--I really enjoyed it. (Shrugs)I liked the fact that this story had a twist to it and it wasn't just two Doms vying for the attention of one woman...,"[didn, t, problems, version, really, enjoyed, shrugs, liked, fact, story, twist, wasn, t, just, two, doms, vying, attention, one, woman, found, intriguing, poor, girl, didn, t, even, know, going, ...",1,0,0.932829
895,"This is how science fiction used to be. Super heroes with super skills, no flaws, just perfect. I am tired of so much reality in my life, I'm tired of reality on TV and movies. Give me some fun an...","[science, fiction, used, super, heroes, super, skills, flaws, just, perfect, tired, much, reality, life, m, tired, reality, tv, movies, give, fun, fantasy, let, pretend, m, ell, m, book, enjoying,...",1,0,0.892482


In [19]:
df_temp['clean_text'].iloc[1]

['since',
 'don',
 't',
 'wi',
 'fi',
 'kindle',
 'usb',
 'cable',
 'lets',
 'download',
 'books',
 'etc',
 'pc',
 'kindle']

In [20]:
#df_final = X_test.copy()
#df_final['label'] = y_test
#df_final['predict'] =  model.predict(X_test)
#df_final.head()

In [21]:
#df_final = df_final.join(df2[['reviewText']]).copy()
#df_final.to_csv('data/emb_results.csv')

In [22]:
df1_test[['reviewText', 'label', 'predict_label']].to_csv('data/final_emb_results.csv')